### 1. Instalación e inicialización de Spark

In [ ]:
# Actualización de los repositorios de UBUNTU
!sudo apt-get update

In [ ]:
# Instalación de JAVA
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# Descarga de SPARK
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz

In [ ]:
# Des-zipeado del instalador
!tar xf spark-2.4.8-bin-hadoop2.7.tgz

In [ ]:
# Instalación de Spark en Python
!pip install -q findspark

In [ ]:
# Configuración de variables de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

In [ ]:
# Inicialización de Spark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


In [ ]:
# Verificación de la sesión de Spark
spark

### 2. Creación de estructura de directorios para las capas LANDING_TMP y LANDING

In [ ]:
# Librería para manipulación del sistema de archivos
import os

In [ ]:
# Supongamos que vamos a ingestar del servidor "VISA" un archivo semi estructurado
#
# - ENTIDAD TRANSACCIONES_BANCARIAS

In [ ]:
# Creamos el directorio asociado al servidor desde donde vamos a ingestar los datos para la capa "LANDING_TMP"
os.mkdir('/content/drive/MyDrive/DATALAKE/LANDING_TMP/VISA')

In [ ]:
# Creamos el directorio para "TRANSACCIONES_BANCARIAS" en "LANDING_TMP"
os.mkdir('/content/drive/MyDrive/DATALAKE/LANDING_TMP/VISA/TRANSACCIONES_BANCARIAS')

In [ ]:
# Creamos el directorio asociado al servidor desde donde vamos a ingestar los datos para la capa "LANDING"
os.mkdir('/content/drive/MyDrive/DATALAKE/LANDING/VISA')

In [ ]:
# Creamos el directorio para "TRANSACCIONES_BANCARIAS" en "LANDING"
os.mkdir('/content/drive/MyDrive/DATALAKE/LANDING/VISA/TRANSACCIONES_BANCARIAS')

### 3. Subir archivos a la capa LANDING_TMP

In [ ]:
# Subimos los datos de TRANSACCIONES_BANCARIAS (el archivo 'transacciones_bancarias.txt') a "LANDING_TMP/VISA/TRANSACCIONES_BANCARIAS"

### 4. Utilizamos Spark para leer datos de la capa LANDING_TMP como dataframe  

In [ ]:
# Leemos los datos como dataframe
dfTransaccionesBancarias = spark.read.format("json").load("/content/drive/MyDrive/DATALAKE/LANDING_TMP/VISA/TRANSACCIONES_BANCARIAS")

# Mostramos los datos
dfTransaccionesBancarias.show()

+--------------+--------------------------+--------------------+
|EMPRESA       |PERSONA                   |TRANSACCION         |
+--------------+--------------------------+--------------------+
|{6, Google}   |{24, 24, Amaya, 1801.0}   |{2018-12-05, 1745.0}|
|{10, Sony}    |{32, 1, Carl, 20095.0}    |{2018-04-19, 238.0} |
|{2, Microsoft}|{34, 65, Nehru, 12423.0}  |{2018-04-19, 4097.0}|
|{5, Amazon}   |{23, 71, Doris, 11538.0}  |{2018-12-05, 1548.0}|
|{5, Amazon}   |{45, 83, Giselle, 2503.0} |{2018-04-19, 2233.0}|
|{4, Toyota}   |{42, 96, Amos, 15855.0}   |{2018-04-19, 2887.0}|
|{9, IBM}      |{70, 19, Laura, 17403.0}  |{2018-04-19, 286.0} |
|{4, Toyota}   |{67, 40, Ross, 14285.0}   |{2018-04-19, 974.0} |
|{8, HP}       |{57, 100, Cynthia, 8682.0}|{2018-04-19, 2698.0}|
|{2, Microsoft}|{22, 22, Kibo, 7449.0}    |{2018-12-05, 1398.0}|
+--------------+--------------------------+--------------------+
only showing top 10 rows



In [ ]:
# Tendremos una estructura de tipos semi estructurada
dfTransaccionesBancarias.printSchema()

root
 |-- EMPRESA: struct (nullable = true)
 |    |-- ID_EMPRESA: string (nullable = true)
 |    |-- NOMBRE_EMPRESA: string (nullable = true)
 |-- PERSONA: struct (nullable = true)
 |    |-- EDAD: long (nullable = true)
 |    |-- ID_PERSONA: string (nullable = true)
 |    |-- NOMBRE_PERSONA: string (nullable = true)
 |    |-- SALARIO: double (nullable = true)
 |-- TRANSACCION: struct (nullable = true)
 |    |-- FECHA: string (nullable = true)
 |    |-- MONTO: double (nullable = true)



### 5. Utilizamos Spark para escribir datos de la capa LANDING_TMP en la capa LANDING

In [ ]:
# Escribimos los datos binarizándolos en la partición de la fecha correspondiente en "LANDING" para "TRANSACCIONES_BANCARIAS"
dfTransaccionesBancarias.write.format("parquet").mode("overwrite").save("/content/drive/MyDrive/DATALAKE/LANDING/VISA/TRANSACCIONES_BANCARIAS/2018-04-19")